# ETL Transform Phase

### Objective
This notebook performs the **Transform** phase of the ETL process.  
Here, the validated datasets from the Extract phase are cleaned, standardized, and enriched to ensure data quality and usability.

Key operations include:
- Handling missing values  
- Standardizing formats and cases  
- Converting data types  
- Creating derived columns  
- Filtering and categorizing data


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Load existing validated datasets
data_dir = Path('data')
validated_full = data_dir / 'validated_full.csv'
validated_inc = data_dir / 'validated_incremental.csv'

if validated_full.exists() and validated_inc.exists():
    full = pd.read_csv(validated_full)
    inc = pd.read_csv(validated_inc)
else:
    # fallback: use raw and incremental, then combine
    raw = pd.read_csv(data_dir / 'raw_data.csv')
    inc = pd.read_csv(data_dir / 'incremental_data.csv')
    full = pd.concat([raw, inc], ignore_index=True)
    full.to_csv(data_dir / 'validated_full.csv', index=False)
    inc.to_csv(data_dir / 'validated_incremental.csv', index=False)

full.shape, inc.shape

((9000, 10), (1000, 10))

### Data Loading

The validated datasets were loaded from the `/data` folder.  
If they were not found, the raw and incremental data were revalidated and merged to ensure continuity.

This step ensures that the Transform phase always begins with a consistent and verified dataset.


In [2]:
# Check missing values
full.isnull().sum()

# Fill missing text columns with "Unknown"
for col in ['region', 'school']:
    if col in full.columns:
        full[col].fillna('Unknown', inplace=True)

C:\Users\nyaga\AppData\Local\Temp\ipykernel_13820\1961318393.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full[col].fillna('Unknown', inplace=True)


### Transformation 1 – Handle Missing Values

Missing entries in the `region` and `school` columns (if any) were filled with `"Unknown"`.  
This prevents null-related issues during aggregation or analysis.

In [3]:
# Standardize casing for selected columns
text_cols = ['gender', 'region', 'subject', 'grade_level']
for col in text_cols:
    if col in full.columns:
        full[col] = full[col].str.title()

### Transformation 2 – Standardize Text Casing

Categorical fields such as `gender`, `region`, and `subject` were standardized to title case (e.g., "Male", "Female").  
This ensures uniform text representation across the dataset.


In [4]:
full['exam_date'] = pd.to_datetime(full['exam_date'], errors='coerce')
full['exam_date'].head()

0   2025-07-02
1   2025-09-09
2   2025-05-10
3   2025-08-12
4   2024-10-21
Name: exam_date, dtype: datetime64[ns]

### Transformation 3 – Convert `exam_date` to Datetime

The `exam_date` column was originally stored as a string (`object`).  
It has been converted into a proper `datetime` data type to allow for accurate time-based filtering and analysis.

In [5]:
full['score_status'] = full['exam_score'].apply(lambda x: 'Pass' if x >= 50 else 'Fail')
full[['exam_score', 'score_status']].head()

,exam_score,score_status
0,48.23,Fail
1,71.79,Pass
2,41.66,Fail
3,56.96,Pass
4,68.71,Pass


### Transformation 4 – Create Derived Column

A new column named `score_status` was added to indicate whether a student passed or failed:
- **Pass:** score ≥ 50  
- **Fail:** score < 50

In [6]:
bins = [0, 50, 70, 85, 100]
labels = ['Low', 'Average', 'Good', 'Excellent']
full['performance_level'] = pd.cut(full['exam_score'], bins=bins, labels=labels, include_lowest=True)
full[['exam_score', 'performance_level']].head()

,exam_score,performance_level
0,48.23,Low
1,71.79,Good
2,41.66,Low
3,56.96,Average
4,68.71,Average


### Transformation 5 – Categorize Scores

Students were grouped into performance levels based on their exam scores:
- **Low:** 0–50  
- **Average:** 50–70  
- **Good:** 70–85  
- **Excellent:** 85–100

In [7]:
if 'name' in full.columns:
    full.drop(columns=['name'], inplace=True)

### Transformation 6 – Drop Irrelevant Columns 

The `name` column was removed since it is not necessary for analysis and may raise privacy concerns.

In [8]:
Path('transformed').mkdir(exist_ok=True)
full.to_csv('transformed/transformed_full.csv', index=False)
inc.to_csv('transformed/transformed_incremental.csv', index=False)

### Saving Transformed Data

Both transformed datasets were exported into the `/transformed` folder as:
- `transformed_full.csv`
- `transformed_incremental.csv`

They are now cleaned, standardized, and enriched for analysis or loading into a warehouse.

## ✅ Transform Phase Summary

| Category | Transformation | Result |
|-----------|----------------|--------|
| Cleaning | Filled missing values | ✅ |
| Standardization | Standardized text case | ✅ |
| Structural | Converted `exam_date` to datetime | ✅ |
| Enrichment | Added `score_status` | ✅ |
| Categorization | Created performance levels | ✅ |
| (Optional) | Dropped name column | ✅ |

The Transform phase successfully cleaned and enhanced the validated data, producing two ready-to-use CSV files for the next ETL step or reporting.
